# Financial and Economic Data Applications

In [1]:
from __future__ import division
from pandas import Series, DataFrame
import pandas as pd
from numpy.random import randn
import numpy as np
pd.options.display.max_rows = 12
np.set_printoptions(precision=4, suppress=True)
import matplotlib.pyplot as plt
plt.rc('figure', figsize=(12, 6))

In [2]:
%matplotlib inline

In [3]:
%pwd

u'/Users/ruiwang/source/pydata-book'

## Data munging topics

### Time series and cross-section alignment

In [4]:
close_px = pd.read_csv('ch11/stock_px.csv', parse_dates=True, index_col=0)
volume = pd.read_csv('ch11/volume.csv', parse_dates=True, index_col=0)
prices = close_px.loc['2011-09-05':'2011-09-14', ['AAPL', 'JNJ', 'SPX', 'XOM']]
volume = volume.loc['2011-09-05':'2011-09-12', ['AAPL', 'JNJ', 'XOM']]

In [5]:
prices

,AAPL,JNJ,SPX,XOM
2011-09-06,379.74,64.64,1165.24,71.15
2011-09-07,383.93,65.43,1198.62,73.65
2011-09-08,384.14,64.95,1185.90,72.82
2011-09-09,377.48,63.64,1154.23,71.01
2011-09-12,379.94,63.59,1162.27,71.84
2011-09-13,384.62,63.61,1172.87,71.65
2011-09-14,389.30,63.73,1188.68,72.64


In [6]:
volume

,AAPL,JNJ,XOM
2011-09-06,18173500.0,15848300.0,25416300.0
2011-09-07,12492000.0,10759700.0,23108400.0
2011-09-08,14839800.0,15551500.0,22434800.0
2011-09-09,20171900.0,17008200.0,27969100.0
2011-09-12,16697300.0,13448200.0,26205800.0


In [7]:
prices * volume

,AAPL,JNJ,SPX,XOM
2011-09-06,6.901205e+09,1.024434e+09,NaN,1.808370e+09
2011-09-07,4.796054e+09,7.040072e+08,NaN,1.701934e+09
2011-09-08,5.700561e+09,1.010070e+09,NaN,1.633702e+09
2011-09-09,7.614489e+09,1.082402e+09,NaN,1.986086e+09
2011-09-12,6.343972e+09,8.551710e+08,NaN,1.882625e+09
2011-09-13,NaN,NaN,NaN,NaN
2011-09-14,NaN,NaN,NaN,NaN


In [8]:
vwap = (prices * volume).sum() / volume.sum()

In [9]:
vwap

AAPL    380.655181
JNJ      64.394769
SPX            NaN
XOM      72.024288
dtype: float64

In [10]:
vwap.dropna()

AAPL    380.655181
JNJ      64.394769
XOM      72.024288
dtype: float64

In [11]:
prices.align(volume, join='inner')

(              AAPL    JNJ    XOM
 2011-09-06  379.74  64.64  71.15
 2011-09-07  383.93  65.43  73.65
 2011-09-08  384.14  64.95  72.82
 2011-09-09  377.48  63.64  71.01
 2011-09-12  379.94  63.59  71.84,
                   AAPL         JNJ         XOM
 2011-09-06  18173500.0  15848300.0  25416300.0
 2011-09-07  12492000.0  10759700.0  23108400.0
 2011-09-08  14839800.0  15551500.0  22434800.0
 2011-09-09  20171900.0  17008200.0  27969100.0
 2011-09-12  16697300.0  13448200.0  26205800.0)

In [12]:
s1 = Series(range(3), index=['a', 'b', 'c'])
s2 = Series(range(4), index=['d', 'b', 'c', 'e'])
s3 = Series(range(3), index=['f', 'a', 'c'])
DataFrame({'one': s1, 'two': s2, 'three': s3})

,one,three,two
a,0.0,1.0,NaN
b,1.0,NaN,1.0
c,2.0,2.0,2.0
d,NaN,NaN,0.0
e,NaN,NaN,3.0
f,NaN,0.0,NaN


In [13]:
DataFrame({'one': s1, 'two': s2, 'three': s3}, index=list('face'))

,one,three,two
f,NaN,0.0,NaN
a,0.0,1.0,NaN
c,2.0,2.0,2.0
e,NaN,NaN,3.0


### Operations with time series of different frequencies

In [21]:
ts1 = Series(np.random.randn(3),
             index=pd.date_range('2012-6-13', periods=3, freq='W-WED'))
ts1

2012-06-13   -0.373310
2012-06-20    0.722502
2012-06-27   -0.100629
Freq: W-WED, dtype: float64

In [22]:
ts1.resample('B')

DatetimeIndexResampler [freq=<BusinessDay>, axis=0, closed=left, label=left, convention=start, base=0]

In [23]:
ts1.resample('B').ffill()

2012-06-13   -0.373310
2012-06-14   -0.373310
2012-06-15   -0.373310
2012-06-18   -0.373310
2012-06-19   -0.373310
2012-06-20    0.722502
2012-06-21    0.722502
2012-06-22    0.722502
2012-06-25    0.722502
2012-06-26    0.722502
2012-06-27   -0.100629
Freq: B, dtype: float64

In [24]:
dates = pd.DatetimeIndex(['2012-6-12', '2012-6-17', '2012-6-18',
                          '2012-6-21', '2012-6-22', '2012-6-29'])
ts2 = Series(np.random.randn(6), index=dates)
ts2

2012-06-12   -0.703655
2012-06-17    0.038055
2012-06-18   -1.110692
2012-06-21   -0.205326
2012-06-22    1.271755
2012-06-29    0.381693
dtype: float64

In [28]:
ts1.reindex(ts2.index, method ='ffill')

2012-06-12         NaN
2012-06-17   -0.373310
2012-06-18   -0.373310
2012-06-21    0.722502
2012-06-22    0.722502
2012-06-29   -0.100629
dtype: float64

In [29]:
ts2 + ts1.reindex(ts2.index, method ='ffill')

2012-06-12         NaN
2012-06-17   -0.335255
2012-06-18   -1.484002
2012-06-21    0.517176
2012-06-22    1.994257
2012-06-29    0.281064
dtype: float64

#### Using periods instead of timestamps

In [30]:
gdp = Series([1.78, 1.94, 2.08, 2.01, 2.15, 2.31, 2.46],
             index=pd.period_range('1984Q2', periods=7, freq='Q-SEP'))
infl = Series([0.025, 0.045, 0.037, 0.04],
              index=pd.period_range('1982', periods=4, freq='A-DEC'))
gdp

1984Q2    1.78
1984Q3    1.94
1984Q4    2.08
1985Q1    2.01
1985Q2    2.15
1985Q3    2.31
1985Q4    2.46
Freq: Q-SEP, dtype: float64

In [31]:
infl

1982    0.025
1983    0.045
1984    0.037
1985    0.040
Freq: A-DEC, dtype: float64

In [32]:
infl_q = infl.asfreq('Q-SEP', how='end')

In [33]:
infl_q

1983Q1    0.025
1984Q1    0.045
1985Q1    0.037
1986Q1    0.040
Freq: Q-SEP, dtype: float64

In [36]:
infl_q.reindex(gdp.index, method='ffill')

1984Q2    0.045
1984Q3    0.045
1984Q4    0.045
1985Q1    0.037
1985Q2    0.037
1985Q3    0.037
1985Q4    0.037
Freq: Q-SEP, dtype: float64

### Time of day and "as of" data selection

In [37]:
# Make an intraday date range and time series
rng = pd.date_range('2012-06-01 09:30', '2012-06-01 15:59', freq='T')
# Make a 5-day series of 9:30-15:59 values
rng = rng.append([rng + pd.offsets.BDay(i) for i in range(1, 4)])
ts = Series(np.arange(len(rng), dtype=float), index=rng)
ts

2012-06-01 09:30:00       0.0
2012-06-01 09:31:00       1.0
2012-06-01 09:32:00       2.0
2012-06-01 09:33:00       3.0
2012-06-01 09:34:00       4.0
2012-06-01 09:35:00       5.0
                        ...  
2012-06-06 15:54:00    1554.0
2012-06-06 15:55:00    1555.0
2012-06-06 15:56:00    1556.0
2012-06-06 15:57:00    1557.0
2012-06-06 15:58:00    1558.0
2012-06-06 15:59:00    1559.0
Length: 1560, dtype: float64

In [38]:
from datetime import time
ts[time(10, 0)]

2012-06-01 10:00:00      30.0
2012-06-04 10:00:00     420.0
2012-06-05 10:00:00     810.0
2012-06-06 10:00:00    1200.0
dtype: float64

In [39]:
ts.at_time(time(10, 0))

2012-06-01 10:00:00      30.0
2012-06-04 10:00:00     420.0
2012-06-05 10:00:00     810.0
2012-06-06 10:00:00    1200.0
dtype: float64

In [40]:
ts.between_time(time(10, 0), time(10, 1))

2012-06-01 10:00:00      30.0
2012-06-01 10:01:00      31.0
2012-06-04 10:00:00     420.0
2012-06-04 10:01:00     421.0
2012-06-05 10:00:00     810.0
2012-06-05 10:01:00     811.0
2012-06-06 10:00:00    1200.0
2012-06-06 10:01:00    1201.0
dtype: float64

In [41]:
np.random.seed(12346)

In [42]:
# Set most of the time series randomly to NA
indexer = np.sort(np.random.permutation(len(ts))[700:])
irr_ts = ts.copy()
irr_ts[indexer] = np.nan
irr_ts['2012-06-01 09:50':'2012-06-01 10:00']

2012-06-01 09:50:00    20.0
2012-06-01 09:51:00     NaN
2012-06-01 09:52:00    22.0
2012-06-01 09:53:00    23.0
2012-06-01 09:54:00     NaN
2012-06-01 09:55:00    25.0
2012-06-01 09:56:00     NaN
2012-06-01 09:57:00     NaN
2012-06-01 09:58:00     NaN
2012-06-01 09:59:00     NaN
2012-06-01 10:00:00     NaN
dtype: float64

In [43]:
selection = pd.date_range('2012-06-01 10:00', periods=4, freq='B')
irr_ts.asof(selection)

2012-06-01 10:00:00      25.0
2012-06-04 10:00:00     420.0
2012-06-05 10:00:00     810.0
2012-06-06 10:00:00    1197.0
Freq: B, dtype: float64

### Splicing together data sources

In [44]:
data1 = DataFrame(np.ones((6, 3), dtype=float),
                  columns=['a', 'b', 'c'],
                  index=pd.date_range('6/12/2012', periods=6))
data2 = DataFrame(np.ones((6, 3), dtype=float) * 2,
                  columns=['a', 'b', 'c'],
                  index=pd.date_range('6/13/2012', periods=6))
spliced = pd.concat([data1.loc[:'2012-06-14'], data2.loc['2012-06-15':]])
spliced

,a,b,c
2012-06-12,1.0,1.0,1.0
2012-06-13,1.0,1.0,1.0
2012-06-14,1.0,1.0,1.0
2012-06-15,2.0,2.0,2.0
2012-06-16,2.0,2.0,2.0
2012-06-17,2.0,2.0,2.0
2012-06-18,2.0,2.0,2.0


In [45]:
data2 = DataFrame(np.ones((6, 4), dtype=float) * 2,
                  columns=['a', 'b', 'c', 'd'],
                  index=pd.date_range('6/13/2012', periods=6))
spliced = pd.concat([data1.loc[:'2012-06-14'], data2.loc['2012-06-15':]])
spliced

,a,b,c,d
2012-06-12,1.0,1.0,1.0,NaN
2012-06-13,1.0,1.0,1.0,NaN
2012-06-14,1.0,1.0,1.0,NaN
2012-06-15,2.0,2.0,2.0,2.0
2012-06-16,2.0,2.0,2.0,2.0
2012-06-17,2.0,2.0,2.0,2.0
2012-06-18,2.0,2.0,2.0,2.0


In [46]:
spliced_filled = spliced.combine_first(data2)
spliced_filled

,a,b,c,d
2012-06-12,1.0,1.0,1.0,NaN
2012-06-13,1.0,1.0,1.0,2.0
2012-06-14,1.0,1.0,1.0,2.0
2012-06-15,2.0,2.0,2.0,2.0
2012-06-16,2.0,2.0,2.0,2.0
2012-06-17,2.0,2.0,2.0,2.0
2012-06-18,2.0,2.0,2.0,2.0


In [47]:
spliced.update(data2, overwrite=False)

In [48]:
spliced

,a,b,c,d
2012-06-12,1.0,1.0,1.0,NaN
2012-06-13,1.0,1.0,1.0,2.0
2012-06-14,1.0,1.0,1.0,2.0
2012-06-15,2.0,2.0,2.0,2.0
2012-06-16,2.0,2.0,2.0,2.0
2012-06-17,2.0,2.0,2.0,2.0
2012-06-18,2.0,2.0,2.0,2.0


In [49]:
cp_spliced = spliced.copy()
cp_spliced[['a', 'c']] = data1[['a', 'c']]
cp_spliced

,a,b,c,d
2012-06-12,1.0,1.0,1.0,NaN
2012-06-13,1.0,1.0,1.0,2.0
2012-06-14,1.0,1.0,1.0,2.0
2012-06-15,1.0,2.0,1.0,2.0
2012-06-16,1.0,2.0,1.0,2.0
2012-06-17,1.0,2.0,1.0,2.0
2012-06-18,NaN,2.0,NaN,2.0


### Return indexes and cumulative returns

In [ ]:
# import pandas.io.data as web
# price = web.get_data_yahoo('AAPL', '2011-01-01')['Adj Close']
# price[-5:]

In [ ]:
# price['2011-10-03'] / price['2011-3-01'] - 1

In [ ]:
# returns = price.pct_change()
# ret_index = (1 + returns).cumprod()
# ret_index[0] = 1  # Set first value to 1
# ret_index

In [ ]:
# m_returns = ret_index.resample('BM', how='last').pct_change()
# m_returns['2012']

In [ ]:
# m_rets = (1 + returns).resample('M', how='prod', kind='period') - 1
# m_rets['2012']

In [ ]:
# returns[dividend_dates] += dividend_pcts

## Group transforms and analysis

In [50]:
pd.options.display.max_rows = 100
pd.options.display.max_columns = 10
np.random.seed(12345)

In [51]:
import random; random.seed(0)
import string

N = 1000
def rands(n):
    choices = string.ascii_uppercase
    return ''.join([random.choice(choices) for _ in xrange(n)])
tickers = np.array([rands(5) for _ in xrange(N)])

In [52]:
M = 500
df = DataFrame({'Momentum' : np.random.randn(M) / 200 + 0.03,
                'Value' : np.random.randn(M) / 200 + 0.08,
                'ShortInterest' : np.random.randn(M) / 200 - 0.02},
                index=tickers[:M])

In [53]:
ind_names = np.array(['FINANCIAL', 'TECH'])
sampler = np.random.randint(0, len(ind_names), N)
industries = Series(ind_names[sampler], index=tickers,
                    name='industry')

In [54]:
by_industry = df.groupby(industries)
by_industry.mean()

,Momentum,ShortInterest,Value
industry,,,
FINANCIAL,0.029485,-0.020739,0.079929
TECH,0.030407,-0.019609,0.080113


In [55]:
by_industry.describe()

Momentum                                            ...     \
             count      mean       std       min       25%    ...      
industry                                                      ...      
FINANCIAL    246.0  0.029485  0.004802  0.017210  0.026263    ...      
TECH         254.0  0.030407  0.005303  0.016778  0.026456    ...      

              Value                                          
                min       25%       50%       75%       max  
industry                                                     
FINANCIAL  0.067025  0.076638  0.079804  0.082718  0.093334  
TECH       0.065253  0.076737  0.080296  0.083353  0.093081  

[2 rows x 24 columns]

In [56]:
# Within-Industry Standardize
def zscore(group):
    return (group - group.mean()) / group.std()

df_stand = by_industry.apply(zscore)

In [57]:
df_stand.groupby(industries).agg(['mean', 'std'])

Momentum      ShortInterest              Value     
                   mean  std          mean  std          mean  std
industry                                                          
FINANCIAL  1.114736e-15  1.0  3.081772e-15  1.0  8.001278e-15  1.0
TECH      -2.779929e-16  1.0 -1.910982e-15  1.0 -7.139521e-15  1.0

In [58]:
# Within-industry rank descending
ind_rank = by_industry.rank(ascending=False)
ind_rank.groupby(industries).agg(['min', 'max'])

Momentum        ShortInterest        Value       
               min    max           min    max   min    max
industry                                                   
FINANCIAL      1.0  246.0           1.0  246.0   1.0  246.0
TECH           1.0  254.0           1.0  254.0   1.0  254.0

In [59]:
# Industry rank and standardize
by_industry.apply(lambda x: zscore(x.rank()))

,Momentum,ShortInterest,Value
VTKGN,-0.091346,-0.976696,-1.004802
KUHMP,0.794005,1.299919,-0.358356
XNHTQ,-0.541047,-0.836164,-1.679355
GXZVX,-0.583207,-1.623142,0.990749
ISXRM,1.572120,-0.265423,0.374314
CLPXZ,1.395171,0.360703,-1.436006
MWGUO,-0.074863,0.632931,-0.660154
ASKVR,0.238200,-0.319869,0.102086
AMWGI,1.075068,1.665302,1.580983
WEOGZ,1.412345,0.358356,-1.313972


### Group factor exposures

In [73]:
from numpy.random import rand

fac1, fac2, fac3 = np.random.rand(3, 1000)

ticker_subset = tickers.take(np.random.permutation(N)[:1000])

# Weighted sum of factors plus noise
port = Series(0.7 * fac1 - 1.2 * fac2 + 0.3 * fac3 + rand(1000),
              index=ticker_subset)
factors = DataFrame({'f1': fac1, 'f2': fac2, 'f3': fac3},
                    index=ticker_subset)

In [74]:
factors.corrwith(port)

f1    0.399181
f2   -0.672288
f3    0.214145
dtype: float64

In [75]:
# pd.ols(y=port, x=factors).beta

AttributeError: 'module' object has no attribute 'ols'

In [ ]:
# def beta_exposure(chunk, factors=None):
#     return pd.ols(y=chunk, x=factors).beta

In [76]:
# by_ind = port.groupby(industries)
# exposures = by_ind.apply(beta_exposure, factors=factors)
# exposures.unstack()

### Decile and quartile analysis

In [ ]:
# import pandas.io.data as web
# data = web.get_data_yahoo('SPY', '2006-01-01')
# data.info()

In [ ]:
# px = data['Adj Close']
# returns = px.pct_change()

# def to_index(rets):
#     index = (1 + rets).cumprod()
#     first_loc = max(index.index.get_loc(index.idxmax()) - 1, 0)
#     index.values[first_loc] = 1
#     return index

# def trend_signal(rets, lookback, lag):
#     signal = pd.rolling_sum(rets, lookback, min_periods=lookback - 5)
#     return signal.shift(lag)

In [ ]:
# signal = trend_signal(returns, 100, 3)
# trade_friday = signal.resample('W-FRI').resample('B', fill_method='ffill')
# trade_rets = trade_friday.shift(1) * returns
# trade_rets = trade_rets[:len(returns)]

In [ ]:
# to_index(trade_rets).plot()

In [ ]:
# vol = pd.rolling_std(returns, 250, min_periods=200) * np.sqrt(250)

# def sharpe(rets, ann=250):
#     return rets.mean() / rets.std()  * np.sqrt(ann)

In [ ]:
# cats = pd.qcut(vol, 4)
# print('cats: %d, trade_rets: %d, vol: %d' % (len(cats), len(trade_rets), len(vol)))

In [ ]:
# trade_rets.groupby(cats).agg(sharpe)

## More example applications

### Signal frontier analysis

In [77]:
# names = ['AAPL', 'GOOG', 'MSFT', 'DELL', 'GS', 'MS', 'BAC', 'C']
# def get_px(stock, start, end):
#     return web.get_data_yahoo(stock, start, end)['Adj Close']
# px = DataFrame({n: get_px(n, None, None) for n in names})

NameError: global name 'web' is not defined

In [ ]:
#px = pd.read_csv('ch11/stock_px.csv')

In [ ]:
# plt.close('all')

In [ ]:
# px = px.asfreq('B').fillna(method='pad')
# rets = px.pct_change()
# ((1 + rets).cumprod() - 1).plot()

In [ ]:
# def calc_mom(price, lookback, lag):
#     mom_ret = price.shift(lag).pct_change(lookback)
#     ranks = mom_ret.rank(axis=1, ascending=False)
#     demeaned = ranks.subtract(ranks.mean(axis=1), axis=0)
#     return demeaned.divide(demeaned.std(axis=1), axis=0)

In [ ]:
# compound = lambda x : (1 + x).prod() - 1
# daily_sr = lambda x: x.mean() / x.std()

# def strat_sr(prices, lb, hold):
#     # Compute portfolio weights
#     freq = '%dB' % hold
#     port = calc_mom(prices, lb, lag=1)

#     daily_rets = prices.pct_change()

#     # Compute portfolio returns
#     port = port.shift(1).resample(freq, how='first')
#     returns = daily_rets.resample(freq, how=compound)
#     port_rets = (port * returns).sum(axis=1)

#     return daily_sr(port_rets) * np.sqrt(252 / hold)

In [ ]:
# strat_sr(px, 70, 30)

In [ ]:
# from collections import defaultdict

# lookbacks = range(20, 90, 5)
# holdings = range(20, 90, 5)
# dd = defaultdict(dict)
# for lb in lookbacks:
#     for hold in holdings:
#         dd[lb][hold] = strat_sr(px, lb, hold)

# ddf = DataFrame(dd)
# ddf.index.name = 'Holding Period'
# ddf.columns.name = 'Lookback Period'

In [ ]:
# import matplotlib.pyplot as plt

# def heatmap(df, cmap=plt.cm.gray_r):
#     fig = plt.figure()
#     ax = fig.add_subplot(111)
#     axim = ax.imshow(df.values, cmap=cmap, interpolation='nearest')
#     ax.set_xlabel(df.columns.name)
#     ax.set_xticks(np.arange(len(df.columns)))
#     ax.set_xticklabels(list(df.columns))
#     ax.set_ylabel(df.index.name)
#     ax.set_yticks(np.arange(len(df.index)))
#     ax.set_yticklabels(list(df.index))
#     plt.colorbar(axim)

In [ ]:
# heatmap(ddf)

### Future contract rolling

In [ ]:
# pd.options.display.max_rows = 10

In [ ]:
# import pandas.io.data as web
# Approximate price of S&P 500 index
# px = web.get_data_yahoo('SPY')['Adj Close'] * 10
# px

In [ ]:
# from datetime import datetime
# expiry = {'ESU2': datetime(2012, 9, 21),
#           'ESZ2': datetime(2012, 12, 21)}
# expiry = Series(expiry).order()

In [ ]:
# expiry

In [ ]:
# np.random.seed(12347)
# N = 200
# walk = (np.random.randint(0, 200, size=N) - 100) * 0.25
# perturb = (np.random.randint(0, 20, size=N) - 10) * 0.25
# walk = walk.cumsum()

# rng = pd.date_range(px.index[0], periods=len(px) + N, freq='B')
# near = np.concatenate([px.values, px.values[-1] + walk])
# far = np.concatenate([px.values, px.values[-1] + walk + perturb])
# prices = DataFrame({'ESU2': near, 'ESZ2': far}, index=rng)

In [ ]:
# prices.tail()

In [ ]:
# def get_roll_weights(start, expiry, items, roll_periods=5):
#     # start : first date to compute weighting DataFrame
#     # expiry : Series of ticker -> expiration dates
#     # items : sequence of contract names

#     dates = pd.date_range(start, expiry[-1], freq='B')
#     weights = DataFrame(np.zeros((len(dates), len(items))),
#                         index=dates, columns=items)

#     prev_date = weights.index[0]
#     for i, (item, ex_date) in enumerate(expiry.iteritems()):
#         if i < len(expiry) - 1:
#             weights.ix[prev_date:ex_date - pd.offsets.BDay(), item] = 1
#             roll_rng = pd.date_range(end=ex_date - pd.offsets.BDay(),
#                                      periods=roll_periods + 1, freq='B')

#             decay_weights = np.linspace(0, 1, roll_periods + 1)
#             weights.ix[roll_rng, item] = 1 - decay_weights
#             weights.ix[roll_rng, expiry.index[i + 1]] = decay_weights
#         else:
#             weights.ix[prev_date:, item] = 1

#         prev_date = ex_date

#     return weights

In [ ]:
# weights = get_roll_weights('6/1/2012', expiry, prices.columns)
# weights.ix['2012-09-12':'2012-09-21']

In [ ]:
# rolled_returns = (prices.pct_change() * weights).sum(1)

### Rolling correlation and linear regression

In [ ]:
# aapl = web.get_data_yahoo('AAPL', '2000-01-01')['Adj Close']
# msft = web.get_data_yahoo('MSFT', '2000-01-01')['Adj Close']

# aapl_rets = aapl.pct_change()
# msft_rets = msft.pct_change()

In [ ]:
# plt.figure()

In [ ]:
# pd.rolling_corr(aapl_rets, msft_rets, 250).plot()

In [ ]:
# plt.figure()

In [ ]:
# model = pd.ols(y=aapl_rets, x={'MSFT': msft_rets}, window=250)
# model.beta

In [ ]:
# model.beta['MSFT'].plot()